# 4. Tools and Agents

## Syllabus
* 4.1 Introduction to Tools
* 4.2 Introduction to Agents
* 4.3 Built-in Tools in LangChain
* 4.4 Agents in Action
* 4.5 Building a Multi-Tool Agent
* 4.6 Creating Custom Tools

### 4.1 Introduction to Tools
A tool in LangChain is simply a function that LLM uses when it cannot do on its own.
eg. A calculator is a tool that an LLM can use because LLM's can make math mistakes

Every tool has 3 main parts
* Name: Unique identifier used by the agent
* Description: Natural language instruction so the agent knows when to use it
* Function: Actual python function

## 4.2 Built-in Tools in LangChain

#### Tool Example 1: Calculator Tool

* `PythonREPLTool()` gives the LLM access to a Python REPL(read, eval, print, loop)
* It executes the given string as a python code
* creates a small sandbox python interpretor
* does not require an LLM


In [1]:
import os
import sys
from pathlib import Path

sys.path.append(os.path.abspath(".."))

In [2]:
# initializing the llm
from llm.load_llm import initialize_llm

llm = initialize_llm()

LLM ready: ChatGoogleGenerativeAI


`pip install langchain-experimental`

In [ ]:
from langchain_experimental.tools import PythonREPLTool

calculator = PythonREPLTool()

# Add print() to see the results
print(calculator.run("print(5*12+(33/3))"))
print(calculator.run("print(max(25,69))"))
print(calculator.run("print(sum([i for i in range(1,11)]))"))

71.0

69

55



`LLMMathChain`
* Instead of directly executing code, it asks the LLM to write math as python code, then evaluate it.

In [10]:
from langchain.chains import LLMMathChain

math_chain = LLMMathChain.from_llm(llm = llm, verbose = True)
print(math_chain.run("What is (5*12+(33/3))"))

C:\Users\amalv\AppData\Local\Temp\ipykernel_25156\943593853.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(math_chain.run("What is (5*12+(33/3))"))




> Entering new LLMMathChain chain...
What is (5*12+(33/3))```text
(5*12+(33/3))
```
...numexpr.evaluate("(5*12+(33/3))")...
Answer: 71.0
> Finished chain.
Answer: 71.0


#### Tool Example 2: Search Tool
* LLM's don't have access to fresh information like daily news, stock prices, weather etc
* To fix this, we connect them to a search API.

* There are a few tools for searching
    - Tavily Search API
    - SerpAPI
    - DuckDuckGoSearch

`pip install tavily-python`

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()
tavily_key = os.environ.get("TAVILY_API_KEY")

In [26]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results = 2)

results = search.run("What is the latest weather in Kochi, Kerala, India")
print(results)

C:\Users\amalv\AppData\Local\Temp\ipykernel_20944\1064741316.py:3: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search = TavilySearchResults(max_results = 2)


[{'title': 'Kochi, Kerala, India Monthly Weather - AccuWeather', 'url': 'https://www.accuweather.com/en/in/kochi/204289/september-weather/204289', 'content': '# Kochi, Kerala\n\nKochi\n\nKerala\n\n## Around the Globe\n\nAround the Globe\n\n### Hurricane Tracker\n\n### Severe Weather\n\n### Radar & Maps\n\n### News & Features\n\n### Astronomy\n\n### Business\n\n### Climate\n\n### Health\n\n### Recreation\n\n### Sports\n\n### Travel\n\n### Warnings\n\n### Data Suite\n\n### Forensics\n\n### Advertising\n\n### Superior Accuracy™\n\n### Video\n\n## Monthly\n\n## September\n\n## 2025\n\n## Daily\n\n## Temperature Graph\n\n## Further Ahead\n\nFurther Ahead\n\n### October 2025\n\n### November 2025\n\n### December 2025 [...] ## Around the Globe\n\nAround the Globe\n\n### Hurricane Tracker\n\n### Severe Weather\n\n### Radar & Maps\n\n### News\n\n### Video\n\nTop Stories\n\nHurricane\n\nTropical activity to ramp up before peak of Atlantic hurricane season\n\n5 hours ago\n\nWeather Forecasts\n\nFa

In [4]:
import json

print(json.dumps(results , indent=2))


NameError: name 'results' is not defined

In [17]:
print(results[0]["content"])

We recognize our responsibility to use data and technology for good. We may use or share your data with our data vendors. Take control of your data.

The Weather Channel is the world's most accurate forecaster according to ForecastWatch, Global and Regional Weather Forecast Accuracy Overview, 2021-2024, commissioned by The Weather Company.

Weather Channel

© The Weather Company, LLC 2025 [...] ## Day

A few showers this morning with overcast skies during the afternoon hours. Hazy. High 84F. Winds NW at 10 to 15 mph. Chance of rain 40%.

## Night

Considerable cloudiness. Occasional rain showers after midnight. Hazy. Low 76F. Winds NW at 5 to 10 mph. Chance of rain 50%.

## Wed 03

## Day

Rain showers early with overcast skies later in the day. Hazy. High 83F. Winds NW at 10 to 15 mph. Chance of rain 60%.

## Night [...] ## Recent Locations

## Weather Forecasts

## Radar & Maps

## News & Media

## Products & Account

## Lifestyle

### Specialty Forecasts

# Kochi, Kerala, India

## 

* Alternatively we can use `DuckDuckGo search` 

### Using Search in an Agent
📌 Here’s what happens:
- LLM sees the question.
- Decides → “I don’t know Bitcoin’s price, I must use search.”
- Calls search tool → fetches live price.
- Passes that result into the calculator.
- Returns final answer.

In [23]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, AgentType

# Give the agent tools (search + calculator)
tools = [search, calculator]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Now the LLM can decide when to search
agent.run("What is the current price of Bitcoin in USD divided by 3?")


C:\Users\amalv\AppData\Local\Temp\ipykernel_25156\1698668514.py:7: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new AgentExecutor chain...
Thought: I need to find the current price of Bitcoin in USD, then divide it by 3.  I'll use a search engine to get the current price.

Action: tavily_search_results_json
Action Input: "current Bitcoin price USD"
Observation: [{'title': 'Bitcoin price today, BTC to USD live price, marketcap and chart', 'url': 'https://coinmarketcap.com/currencies/bitcoin/', 'content': 'The live Bitcoin price today is $110,304 USD with a 24-hour trading volume of $67,781,659,091 USD. We update our BTC to USD price in real-time. Bitcoin is up 2.63% in the last 24 hours. The current CoinMarketCap ranking is #1, with a live market cap of $2,196,700,158,553 USD. It has a circulating supply of 19,914,996 BTC coins and a max. supply of 21,000,000 BTC coins. [...] Thanks to its pioneering nature, BTC remains at the top of this energetic market after over a decade of existence. Even after Bitcoin has lost its undisputed dominance, it remains the largest cryptocurrency, wit

'$36768.0'

## 4.3 Creating Custom Tools

In [24]:
from langchain.tools import tool

# define a custom tool with a decorator function
@tool
def get_weather(city: str) -> str:
    """ Get the weather for a given city from these dummy data"""
    weather_data = {
        "New York": "Sunny, 25°C",
        "London": "Rainy, 15°C",
        "Tokyo": "Cloudy, 20°C"
    }

    return weather_data[city]

using it in an agent

In [26]:
tools = [get_weather]

agent2 = initialize_agent(
    tools = tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose= True
)

agent2.run("Whats the weather in London")



> Entering new AgentExecutor chain...
Thought: I need to get the weather for London.  I will use the `get_weather` function.
Action: get_weather
Action Input: London
Observation: Rainy, 15°C
Thought:Thought: I now know the final answer
Final Answer: The weather in London is rainy and 15°C.

> Finished chain.


'The weather in London is rainy and 15°C.'

In [5]:
import numpy as np
from langchain.tools import tool

@tool
def matrix_determinant(matrix: str)->str:
    """
    Calculates the determinant of a 2x2 matrix
    Input: 2x2 matrix in string format
    Output: determinant value in string
    """
    arr = np.array(eval(matrix))
    det = np.linalg.det(arr)
    return f"The determinant is {det}"

In [7]:
from langchain.agents import initialize_agent, AgentType

tools3 = [matrix_determinant]
agent3 = initialize_agent(
    llm = llm,
    tools = tools3,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True
)

agent3.run("Determine the determinant of matrix [[1,2],[3,4]]")

C:\Users\amalv\AppData\Local\Temp\ipykernel_20944\968028663.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent3 = initialize_agent(
C:\Users\amalv\AppData\Local\Temp\ipykernel_20944\968028663.py:11: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent3.run("Determine the determinant of matrix [[1,2



> Entering new AgentExecutor chain...
Thought: I need to calculate the determinant of the given 2x2 matrix.  I can use the `matrix_determinant` function for this.
Action: matrix_determinant
Action Input: "[[1,2],[3,4]]"
Observation: The determinant is -2.0000000000000004
Thought:Thought: I now know the final answer
Final Answer: -2

> Finished chain.


'-2'

## 4.4 Introduction to Agents

An agent is like an AI decision maker that
* takes in a user query as input
* Decides how to solve it (tool or reasoning)
* uses the chosen tools
* returns the final answer

eg. What is the square root of population of Japan.
* Pick at tool:
    - Search tool to get population
    - Calculator tool to get square root
    - Database tool

#### ReAct Framework (Reason + Act)
* The agent reasons step by step
* Then it acts by using tools.

eg. for previous example
* Reason: I don't know the population of Japan?
* Act: Use the search tool
* Observation: 125M
* Reason: I need to calculate the root of 125M
* Act: use the calculator
* Observation: result is 11180
* Response

#### Agent Execution Loop

This is the core cycle of how agents work:

1. Thought → The agent reasons internally. ("I need Japan’s population.")
2. Action → Calls a tool. ("Search for Japan population.")
3. Observation → Gets result from tool. ("125M.")
4. Thought → Decides next step. ("Now calculate the square root.")
5. Action → Uses calculator.
6. Observation → Gets the number.
7. Response → Returns final answer to user.

👉 This loop continues until the agent has enough info to answer.

## 4.5 Agents in Action

#### Zero-Shot ReAct Description Agent

* Zero-Shot → means the agent hasn’t been given examples ahead of time.
* ReAct → means it uses the Reason + Act loop we discussed.
* Description → it decides which tool to use based only on the natural language description of tools.

👉 Example tools given to the agent:

Calculator: "Useful for math operations like addition, subtraction, multiplication, division, or square roots."

Search: "Useful for finding real-world facts from the internet."

* Lets build a small agent that takes in names of some countries
* Finds their population
* Square roots them and store it in csv file

In [28]:
from langchain.tools import tool
import math
import csv
# Calculator Tool
@tool
def square_root_tool(population: str) -> str:
    """
    This tool takes in a population as string and returns its square root in string format
    """
    value = int(population)
    return str(math.sqrt(value))

@tool
def write_to_csv(row: str) -> None:
    """
    This tool takes a row as string and writes it into a csv file.
    The input must be of the form city,population,square_root_of_population.
    eg: "Tokyo 125000000 11180"
    """
    item = row.split(" ")
    file = open("output.csv" , "a+")
    writer = csv.writer(file)
    writer.writerow(item)
    file.close()

@tool
def read_from_csv(dummy: str = "read") -> str:
    """
    This tool reads rows from the csv file and returns them as a string.
    Input is ignored (just pass anything).
    """
    with open("output.csv", "r") as file:
        data = file.read()
    return data

creating an agent that will take a couple of cities, find its population, find it square root and save it into csv file

In [29]:
agent4 = initialize_agent(
    tools = [search, square_root_tool , write_to_csv, read_from_csv],
    llm = llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True
)

cities = "London, Tokyo, New York"
agent4.run(f"You are an agent that will fetch population count of these cities {cities}. After finding the population, find the square root of each city and write it into the csv file in the format city,population,root. Display the final csv file.")



> Entering new AgentExecutor chain...
Thought: I need to find the population of London, Tokyo, and New York.  I'll use a search engine for that. Then I'll calculate the square root of each population and write the city, population, and square root to a CSV file. Finally, I'll read and display the contents of the CSV file.

Action: tavily_search_results_json
Action Input: "population of London, population of Tokyo, population of New York"
Observation: [{'title': 'London Population 2025', 'url': 'https://worldpopulationreview.com/cities/united-kingdom/london', 'content': "The first half of the 20th century saw sustained, and fairly rapid growth and London’s population reached its highest point in 1939. By the outbreak of the second world war, 8,615,245 people were living in London, although by then it had just lost its status as the largest city in the world to New York. [...] London's 2025 population is now estimated at 9,840,740. In 1950, the population of London was 8,360,850. Londo

'The CSV file contains the following data, but there\'s a formatting issue with the "New York" entry:\n\nLondon,9841000,3137.036818400447\nTokyo,37036200,6085.737424503295\nNew,"York,8478072,2911.712897934822\n\nThe "New York" row should read "New York,8478072,2911.712897934822"'

## 4.6 Agent Types

Got it 👍 Here’s a **clean Markdown reference** for all the main LangChain Agent Types. You can drop this straight into your notebook (`004_agents.md`) as documentation.

---

# 🧠 LangChain Agent Types

LangChain provides different **agent types**, each with its own reasoning style, memory handling, and tool support. Agents differ in **how they decide when and how to use tools**.

---

## 1. Zero-Shot ReAct Description

**Description**

* Uses the **ReAct (Reason + Act)** framework.
* LLM decides which tool to call based only on tool descriptions.
* No prior examples (“zero-shot”).

**When to Use**

* General-purpose agents.
* Tasks where the model needs to reason step by step and use tools.

**Limitations**

* Supports only **single-input tools** (`str -> str`).
* No memory between calls.

**Example**

```python
agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
```

---

## 2. Conversational ReAct Description

**Description**

* Same as Zero-Shot ReAct, but includes **memory** of the conversation.
* Lets you ask follow-up questions without repeating context.

**When to Use**

* Chatbots and assistants that need dialogue memory.

**Limitations**

* Still only works with **single-input tools**.

---

## 3. Self-Ask with Search

**Description**

* Agent decomposes a query into **sub-questions**.
* Uses **search tools** when it cannot answer from reasoning alone.

**When to Use**

* Knowledge Q\&A.
* Tasks where an LLM must fetch external information.

**Limitations**

* Not general-purpose — designed specifically for search-based reasoning.

---

## 4. Plan-and-Execute

**Description**

* Works in **two stages**:

  1. LLM **plans** the full set of steps.
  2. Then **executes** each step with sub-agents.

**When to Use**

* Complex, multi-step workflows.
* Scenarios requiring structured execution instead of ad-hoc reasoning.

**Limitations**

* Can be slower (planning + execution).
* Sometimes “over-plans.”

---

## 5. Structured Chat Agent

**Description**

* Unlike Zero-Shot, it supports **multi-input tools**.
* LLM calls tools with structured arguments (like a JSON dict).

**When to Use**

* When tools need multiple parameters (e.g., `{city: "Tokyo", population: 125000000}`).

**Limitations**

* Requires stricter prompts and well-defined tool schemas.

---

## 6. Functions Agent (OpenAI/Gemini)

**Description**

* Uses **function-calling APIs** of LLMs (OpenAI, Gemini, etc.).
* LLM outputs valid JSON with function arguments.
* Very reliable for tool use.

**When to Use**

* If your LLM supports function calling (e.g., GPT-4, Gemini).
* Production-grade tool invocation.

**Limitations**

* Dependent on LLM providers that implement function calling.

---

# 📊 Summary Table

| Agent Type                      | Multi-Input Tools? | Memory? | Best For                  |
| ------------------------------- | ------------------ | ------- | ------------------------- |
| Zero-Shot ReAct Description     | ❌ No               | ❌ No    | General tool use          |
| Conversational ReAct            | ❌ No               | ✅ Yes   | Chat assistants           |
| Self-Ask with Search            | ❌ No               | ❌ No    | Knowledge Q\&A            |
| Plan-and-Execute                | ❌ No               | ❌ No    | Complex workflows         |
| Structured Chat Agent           | ✅ Yes              | ❌ No    | Multi-input tools         |
| Functions Agent (OpenAI/Gemini) | ✅ Yes              | ❌ No    | Reliable structured calls |

---

⚡ **Tip**: Start with **Zero-Shot ReAct** (simplest), then move to **Structured Chat** or **Functions Agent** once your tools get more complex.

